In [1]:
# change to the root directory of the project
%cd ..
%pip install -e .

/home/ubuntu/code/decentralized-search


Obtaining file:///home/ubuntu/code/decentralized-search
  Preparing metadata (setup.py) ... done
  Attempting uninstall: openkaito
    Found existing installation: openkaito 0.2.4
    Uninstalling openkaito-0.2.4:
      Successfully uninstalled openkaito-0.2.4
  Running setup.py develop for openkaito
Note: you may need to restart the kernel to use updated packages.


In [2]:
import asyncio
import json
import os
import random
from datetime import datetime, timedelta

import bittensor as bt
import openai
from dotenv import load_dotenv

from openkaito.crawlers.twitter.apidojo import ApiDojoTwitterCrawler
from openkaito.evaluation.evaluator import Evaluator
from openkaito.protocol import StructuredSearchSynapse
from openkaito.tasks import (generate_author_index_task,
                             generate_structured_search_task)
from openkaito.utils.version import get_version

load_dotenv()

True

## Query Miners

In [3]:
wallet = bt.wallet(name="<wallet name>", hotkey="<hotkey name>")
metagraph = bt.metagraph(5, sync=True)

miners_uid = list(range(20))

with open("twitter_usernames.txt") as f:
    twitter_usernames = f.read().strip().splitlines()

# search_query = generate_author_index_task(10, 2)
# search_query.timeout = 90

# search_query = generate_structured_search_task()
search_query = StructuredSearchSynapse(
    query_string="TAO OR Bittensor",
    size=5,
    later_than_timestamp=(datetime.now() - timedelta(days=365)).timestamp(),
    author_usernames=random.sample(twitter_usernames, 1000),
    version=get_version(),
)
# set a short timeout to direct query the miners, forcing them not to crawl data on request
search_query.timeout = 10
# or set a long timeout to allow the miners to crawl data on request
# search_query.timeout = 90
print(search_query.json())

async def send():
    dendrite = bt.dendrite(wallet=wallet)
    responses = await dendrite(
        axons=[metagraph.axons[uid] for uid in miners_uid],
        synapse=search_query,
        deserialize=True,
        timeout=search_query.timeout,
    )
    return responses

responses = asyncio.run(send())
print(json.dumps(responses, indent=2))


2024-04-03 06:57:40.522 |       INFO       | You are connecting to finney network with endpoint wss://entrypoint-finney.opentensor.ai:443.
2024-04-03 06:57:40.523 |     WARNING      | We strongly encourage running a local subtensor node whenever possible. This increases decentralization and resilience of the network.
2024-04-03 06:57:40.523 |     WARNING      | In a future release, local subtensor will become the default endpoint. To get ahead of this change, please run a local subtensor node and point to it.


2024-04-03 06:57:40.921 |       INFO       | Connected to finney network and wss://entrypoint-finney.opentensor.ai:443.
{"name": "StructuredSearchSynapse", "timeout": 10.0, "total_size": 0, "header_size": 0, "dendrite": {"status_code": null, "status_message": null, "process_time": null, "ip": null, "port": null, "version": null, "nonce": null, "uuid": null, "hotkey": null, "signature": null}, "axon": {"status_code": null, "status_message": null, "process_time": null, "ip": null, "port": null, "version": null, "nonce": null, "uuid": null, "hotkey": null, "signature": null}, "computed_body_hash": "", "required_hash_fields": [], "query_string": "TAO OR Bittensor", "size": 5, "earlier_than_timestamp": null, "later_than_timestamp": 1680591485, "author_usernames": ["_BrandonDoyle_", "CryptoGorillaYT", "yelotree", "casperdefi", "alenaxbt", "3gmdev", "nindibabi", "0xfoobar", "porsheonna", "DudespostingWs", "Badmon187", "GiancarloChaux", "redbottomdancer", "cnig69", "GotrillaGorilla", "Captain_

In [4]:

#### Evaluation

# need to setup OPENAI_API_KEY and APIFY_API_KEY in .env
llm_client = openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    organization=os.getenv("OPENAI_ORGANIZATION"),
    max_retries=3,
)

# for integrity check
twitter_crawler = ApiDojoTwitterCrawler(os.environ["APIFY_API_KEY"])

evaluator = Evaluator(llm_client, twitter_crawler)

evaluator.evaluate(search_query, responses)

2024-04-03 06:58:10.749 |       INFO       | loaded 1748 credit_twitter_author_usernames
2024-04-03 06:58:58.702 |       INFO       | LLM ranking: [0, 0.5, 0.5, 1, 0.5]
2024-04-03 06:58:58.703 |       INFO       | Quality score: 0.5            
2024-04-03 06:59:03.143 |       INFO       | LLM ranking: [0.5, 0, 0.5, 0.5, 0]
2024-04-03 06:59:03.144 |       INFO       | Quality score: 0.30000001192092896
2024-04-03 06:59:08.793 |       INFO       | LLM ranking: [0.5, 1, 1, 0, 1]
2024-04-03 06:59:08.794 |       INFO       | Quality score: 0.699999988079071
2024-04-03 06:59:12.072 |       INFO       | LLM ranking: [0, 0.5, 0.5, 1, 0.5]
2024-04-03 06:59:12.073 |       INFO       | Quality score: 0.5            
2024-04-03 06:59:17.228 |       INFO       | LLM ranking: [0.5, 1, 1, 0, 0]
2024-04-03 06:59:17.230 |       INFO       | Quality score: 0.5            
2024-04-03 06:59:20.700 |       INFO       | LLM ranking: [0, 0.5, 0.5, 0, 0.5]
2024-04-03 06:59:20.701 |       INFO       | Quality 

tensor([0.6468, 0.0000, 0.5105, 0.6589, 0.0000, 0.6468, 0.0000, 0.5026, 0.0000,
        0.0000, 0.0000, 0.5153, 0.0000, 0.5525, 0.5142, 0.7371, 0.6161, 0.0000,
        0.5142, 1.0000])